# Natural Language Processing (NLP)-Assisted Meta-Analysis of Diabetes as a Risk Factor for COVID-19 Death
## Challenge: COVID-19 Open Research Dataset Challenge (CORD-19)
## Task: What do we know about COVID-19 risk factors?

## Challenge Description:
### The White House and a coalition of leading research groups have issued a call to action to the world's artificial intelligence experts to develop text and data mining tools that can help the medical community answer high priority scientific questions. The Allen Institute and Google/Kaggle is sponsoring a research challenge that utilizes the COVID-19 Open Research Dataset (CORD-19), which represents the most extensive machine-readable coronavirus literature collection (initially 30,000+ papers) available for data mining to date.

## Problem Statement:
### Risk factors associated with death from COVID-19 are not well understood. As COVID-19 spreads worldwide, results of scientific studies may be differentially impacted by diverse underlying populations, testing, containment strategies, and healthcare systems.

## GDIT Approach and Executive Summary: 
### Approach
#### General Dynamics Information Technology (GDIT) ML/AI specialists conducted Natural Language Processing (NLP) methods to extract relevant scientific papers from the CORD-19 dataset and then applied NLP parsers to extract important statistics from the full text articles creating an analytic dataset for statistical meta-analysis. This dataset was then used to derive a pooled estimate of diabetes prevalence in fatal COVID-19 cases from scientific studies in different populations around the world, contributing to the understanding of the risk of death in COVID-19 patients with diabetes. The process and statistical framework of this notebook can be used to identify the prevalence of other comorbidities in COVID-19 cases and add to the scientific knowledge of risk factors for COVID-19.

### Included Datasets
#### For the first submission of this notebook, the challenge metadata file and all pmc_json and pdf_json files were utilized.  At the time of submission, all code and observations can be replicated.  Future use of this notebook could incur errors if the underlying datasets are changed; which was generally the case each week leading up to the initial submission deadline.

### Results
#### At the time of submission this notebook identified 6 separate scientific studies and automated the extraction, meta-analyisis, and visualization of diabetes prevalence estimates in fatal COVID-19 cases across all studies. 

### Pros and Cons of this Approach
#### Pros: 1) Automated identification and extraction of full text articles greatly reduces time intensive process of meta-analysis, 2) interdisciplinary approach involving NLP, statistics, and data visualization, 3) framework could easily be amended to study COVID-19 and other comorbidities such as hypertension or chronic obstructive pulmonary disease (COPD). 

#### Cons: 1) The tables of full text articles were not included in the extraction resulting in the exclusion of these papers from the meta-analysis (introducing potential for bias), 2) the challenge data comprised of text extracted into json files are often incomplete, inaccurate, and containing headers/footers which negatively impacts NLP approaches.  

### Visualizations:
* [Visualization #1](#viz1): Bar Chart to Understand When Articles Were Published
* [Visualization #2](#viz2): Word Clouds for Exploring Text Field Composition
* [Visualization #3](#viz3): displaCy Utilization for Entity Identification Visualization
* [Visualization #4](#viz4): Diabetes Prevalence in Fatal COVID-19 Cases by Study

### Additional Approaches Tested but not Implemented Below:
* spaCy's web_sm and web_lg pretained models were utilized for entity identification and extraction but it is our assumption that they were not trained using medical journals and were therefore limited in their effectiveness for this specific use case
* Following manual review of subsets of article which resulted in a sample of Y/N's to whether or not the articles were relevant, a tfidf model was created with the gensim python package in an attempt to predict which other articles were of interest.  Without proper full text, including the data from tables, the model lacked the required data to accurately predict acceptable articles (although it did a fair job at predicting those that were not relevant at all to the COVID-19 challenge).
* Additional python packages, such as medaCy and Bert, were researched and tested but they too appeared to lack relevantly trained models and/or the ability to overcome shortcomings in the provided data set to be of use in this phase of the challenge.

## Environment Setup

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandasql import sqldf # sql access to pandas dataframes
sql = lambda q: sqldf(q, globals())
from collections import defaultdict
import json #utilized when parsing the json files associated with the challenge articles
from smart_open import open
import os

from wordcloud import WordCloud, STOPWORDS #For creating/visualizing Word Clouds in the notebook

import spacy #Entity Extraction/Labeling
from spacy.pipeline import EntityRuler,Sentencizer
from spacy.lang.en import English
from spacy import displacy #Utilized to explore entity identification in-line with the documents themselves

import matplotlib #Meta-Analysis Visualizations
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

#Adjust the parameters below, when required, to help review dataframes easier.  
##With a fail amount of the data being long text fields, for example, adjusting the column widths will help when trying to manually review data.
pd.set_option('display.max_colwidth',100)
pd.set_option("display.max_rows", 20)

## Loading the Challenge Metadata file
### The challenge metadata file is often updated and the subsequent dataframe may change. If issues arise when utilizing this notebook in the future, removing and re-adding the dataset may resolve the issues.

In [ ]:
metadata_df = pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv', low_memory=False)

<a id='viz1'></a>
## Data Exploration
### The articles in the challenge dataset span multiple years, here we explore the data to understand when articles were published.

In [ ]:
#Extracting the year from the publish_date field, and then summarizing to better understand the provided dataset
PublishDates=pd.DataFrame(pd.DatetimeIndex(metadata_df['publish_time']).year)
PublishDates_df2=sql("""SELECT * from (select publish_time as year, count(publish_time) as article_count from PublishDates group by publish_time) where article_count >500 """)

In [ ]:
#Creating a simple bar chart to review how many articles are present by year for which they were published
figure(figsize=(15,10))

y_pos = np.arange(len(PublishDates_df2['year']))
frequency = PublishDates_df2['article_count']


plt.bar(y_pos, frequency, align='center', alpha=0.5)
plt.xticks(y_pos, PublishDates_df2['year'])
plt.ylabel('Article Count')
plt.xlabel('Year')
plt.title('Article Count by Published Year')

plt.show()

### The focus of this effort is centered around published COVID-19 statistics, and this chart helped our team understand that we would not need to incorporate all the full text json files at the onset of this process. Subsequent cells in this notebook will address the filters that were applied to simplify the process.

## Identifying Papers Relevant to the Diabetes and Comorbidity for the COVID-19 Meta-Analysis Use Case
### Articles were initially filtered to focus on those published December 2019 to present.

In [ ]:
article_selection_df1 = sql("""
SELECT row_number() OVER () - 1 as id, *
FROM metadata_df 
WHERE abstract is not NULL
AND (has_pdf_parse = True OR has_pmc_xml_parse = True)
AND (pubmed_id >= 31775236
     OR (pubmed_id is null AND (publish_time like '%2020%' or date(publish_time) >='2019-12-01')))
""")
##PubMed IDs were utilized as a means to identify relevant articles since the IDs are sequential AND not all dates in the metadata file are accurate

### We incorporated full text code extraction and article selection based on the full text. This was done before the application of additional filters as the quality and availability of the metadata file abstracts is limited.

In [ ]:
article_selection_df2 = article_selection_df1.assign(sha=article_selection_df1.sha.str.split(';'))
article_selection_df2 = article_selection_df2.explode('sha')
#Difference from the first metadata file to now (changes made 4/3).  Two different paths now for different json files
article_selection_df2_pdf = sql("""
SELECT '../input/CORD-19-research-challenge/' || full_text_file || '/' || full_text_file || '/' || 'pdf_json/' || trim(sha) || '.json' as filename
, *
FROM article_selection_df2
WHERE has_pdf_parse = True
""")
article_selection_df2_pmc = sql("""
SELECT '../input/CORD-19-research-challenge/' || full_text_file || '/' || full_text_file || '/' || 'pmc_json/' || trim(pmcid) || '.xml.json' as filename
, *
FROM article_selection_df2
WHERE has_pmc_xml_parse = True
""")
article_selection_df3=article_selection_df2_pdf.append(article_selection_df2_pmc)

### In this section we derive the file_exists column, which is subsequently filtered on in the SQL statement. This field indicates that the articles have json data available and the paths to the files exist.

In [ ]:
article_selection_df3 = article_selection_df3.assign(file_exists=lambda x: x.filename.apply(os.path.exists))
article_selection_df3 = sql("""
SELECT filename, file_exists, cord_uid, sha, source_x, title, doi, pmcid, pubmed_id, license, abstract, publish_time, authors, journal, full_text_file, url
FROM article_selection_df3
WHERE file_exists = True
""")

### The metadata file attempts to display the abstracts from the articles. After manual review of the file, the related json, and various online articles themselves, it was determined that the field was generally incomplete, unreliable, or nonexistent. To overcome this issue, the following code extracts both the abstracts and the full text from the JSON data.

In [ ]:
filenames = article_selection_df3.filename.values
file_data = ( json.load(open(f, mode='r')) for f in filenames )
docs = {'abstracts': list(), 'full_text': list()}
for jdoc in file_data:
    docs['full_text'].append( ' '.join([ para['text'] for para in jdoc['body_text'] ]) )
    docs['abstracts'].append( ' '.join([ para['text'] for para in jdoc.get('abstract', []) ]) )
article_selection_df4 = article_selection_df3.assign(full_text=docs['full_text'], json_abstract=docs['abstracts'])

<a id='viz2'></a>
## Data Exploration - Adding Filters
### At this point in the process, the dataset in use contains the abstract from the metadata file, the abstract extracted from the json files, and the full text also extracted from the json files.  The goal of the word clouds created below is to better understand which text data should be used for further filtering AND to help identify what the additional filter logic will be.


In [ ]:
#Word Cloud for the full text extracted from the json files
json_full_text = article_selection_df4.full_text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = STOPWORDS).generate(str(json_full_text))
fig = plt.figure(
    figsize = (15, 10),
    facecolor = 'k',
    edgecolor = 'k')
plt.subplot(1,2,1)
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)

#Word Cloud for the abstract text from the metadata file
abstract_text = article_selection_df4.abstract.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = STOPWORDS).generate(str(abstract_text))
fig = plt.figure(
    figsize = (15, 10),
    facecolor = 'k',
    edgecolor = 'k')
plt.subplot(1,2,2)
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)

#A word cloud for the extracted abstract text was not included as it was similar to the results from the metadata file.

#showing both of the word clouds defined above
plt.show()

### At the time of submission of this notebook, one of the most prevalent words in both word clouds is the word human.  This is significant and directly relates to some of the logic applied in the cells below because some of the articles supplied for this challenge relate more generally to human coronaviruses, not specifically COVID-19.  Similarly, as seen in the word cloud created from the abstract text, MERS is a common topic in the supplied articles, and as confirmed in minimal manual review, are not relevant to the use case being addressed by this submission.
### The word clouds are also useful for determining that all of the supplied and derived text fields should be used when further isolating relevant articles as some key words are more common in one versus the other.  For example, in the abstract text word cloud you will notice the prevalence of the word "ICU", which is useful in identifying severe cases of COVID-19; for the full text cloud "ICU" is not prevalent.  In conclusion, based on the results of the word clouds and minimal manual review, the following cell applies additional criteria to all of the text fields.

In [ ]:
article_selection_df5 = sql("""
SELECT filename, file_exists, cord_uid, sha, source_x, title, doi, pmcid, pubmed_id, license, abstract as meta_abs,length(abstract) as meta_abs_len,json_abstract, 
        length(json_abstract) as json_abs_len,publish_time, authors, journal, full_text_file, url, full_text
FROM article_selection_df4 
WHERE ((abstract like '%covid%' or abstract like '%novel coronavirus%'or abstract like '%SARS-COV-2%'or abstract like '%SARS-COV2%'or abstract like '%SARSCOV2%'
    or abstract like '%2019-nCoV%' or abstract like '%2019nCoV%')
    and
    (abstract like '%diabetes%' or abstract like '%diabetic%'or abstract like '%hyperglycemia%'or abstract like '%blood sugar%'or abstract like '%HbA1c%'
    or abstract like '%Hemoglobin A1c%' or abstract like '%heart failure%'or abstract like '%hypertension%'or abstract like '%hypertensive%'or abstract like '%blood pressure%')
    and 
    (abstract like '%death%' or abstract like '%fatality%' or abstract like '%non-survivor%' or abstract like '%nonsurvivor%' or abstract like '%fatal%' or abstract like '%fatalities%'))
    OR
    ((json_abstract like '%covid%' or json_abstract like '%novel coronavirus%'or json_abstract like '%SARS-COV-2%'or json_abstract like '%SARS-COV2%'or json_abstract like '%SARSCOV2%'
    or json_abstract like '%2019-nCoV%' or json_abstract like '%2019nCoV%')
    and
    (json_abstract like '%diabetes%' or json_abstract like '%diabetic%'or json_abstract like '%hyperglycemia%'or json_abstract like '%blood sugar%'or json_abstract like '%HbA1c%'
    or json_abstract like '%Hemoglobin A1c%' or json_abstract like '%heart failure%'or json_abstract like '%hypertension%'or json_abstract like '%hypertensive%'or json_abstract like '%blood pressure%')
    and 
    (json_abstract like '%death%' or json_abstract like '%fatality%' or json_abstract like '%non-survivor%' or json_abstract like '%nonsurvivor%' or json_abstract like '%fatal%' or json_abstract like '%fatalities%'))
    OR 
    ((full_text like '%covid%' or full_text like '%novel coronavirus%'or full_text like '%SARS-COV-2%'or full_text like '%SARS-COV2%'or full_text like '%SARSCOV2%'
    or full_text like '%2019-nCoV%' or full_text like '%2019nCoV%')
    and
    (full_text like '%diabetes%' or full_text like '%diabetic%'or full_text like '%hyperglycemia%'or full_text like '%blood sugar%'or full_text like '%HbA1c%'
    or full_text like '%Hemoglobin A1c%' or full_text like '%heart failure%'or full_text like '%hypertension%'or full_text like '%hypertensive%'or full_text like '%blood pressure%')
    and 
    (full_text like '%death%' or full_text like '%fatality%' or full_text like '%non-survivor%' or full_text like '%nonsurvivor%' or full_text like '%fatal%' or full_text like '%fatalities%'))
""")

### The SQL code below can be used to see how many articles are processed, retained, and/or dropped during the process so far. This can also be useful for those "keeping score at home" each time the code is executed as new articles are added to the challenge dataset frequently.

In [ ]:
ArticleSelectionSummary=sql("""
SELECT InitialCount,DateFilterCount,WithJsonFilterCount,WithFullTextCount,FinalListCount from
(SELECT count(*) as InitialCount from metadata_df) a join 
(SELECT count(*) as DateFilterCount from article_selection_df1) b join
(SELECT count(*) as WithJsonFilterCount from article_selection_df2) c join
(SELECT count(*) as WithFullTextCount from article_selection_df4) d join
(SELECT count(*) as FinalListCount from article_selection_df5) e
""")
#WithJsonFilterCount shows an increase due to splitting those articles listed with multiple sha's; to ensure all relevant data is abstracted in following processes
##WithFullTextCount shows an increase due to the articles having both sets of json data

ArticleSelectionSummary

## Relevant Entity and Statistics Extraction

### Our team took the approach of defining our entities through the use of the spaCy EntityRule capability and the "simple" pretrained English model. Other entity recognition approaches did not fit the needs of our use case as well or did not accommodate the significant variation in layout, format, and use of vernacular within the articles.
### For this submission our team focused on extracting and analyzing the statistics associated with fatalities and the diabetes comorbidity.  As seen in the next cell the first section of patterns, those with the "nFatal" label, search the text data for instances where the number of fatalities were referenced.  The second set of patterns identify the total number of patients/cases described in the article.  Similarly, the last section of "nper" labeled patterns focus on identifying instances in the text where statistics were supplied around the percentage of patients who had diabetes.
### Additional patterns were tested to further categorize the demographics of these patients (case severity, gender, etc.) but without acccess to the data in the article tables, these patterns were not succesful.

In [ ]:
nlp = English()
ruler = EntityRuler(nlp)
nlp.add_pipe(nlp.create_pipe('sentencizer'))
patterns = [{"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'non','OP':'+'}, {'IS_PUNCT': True,'OP':'+'}, {'LOWER': 'survivors','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'fatalities','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'cases','OP':'+'},{'LOWER': 'with','OP':'+'},{'LOWER': 'fatalities','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'cases','OP':'+'},{'LOWER': 'died','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'deaths','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'died','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'fatal','OP':'+'}],'id':'n_fatalities'},
            {"label": "nFatal", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'patients','OP':'+'},{'LOWER': 'who','OP':'+'},{'LOWER': 'died','OP':'+'}],'id':'n_fatalities'},                                         
     
            {"label": "n", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'patients','OP':'+'}],'id':'total patients'},
            {"label": "n", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'and','OP':'+'},{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'patients','OP':'+'}],'id':'total patients'},
            {"label": "n", "pattern": [{'LIKE_NUM': True,'OP':'+'}, {'LOWER': 'cases','OP':'+'}],'id':'total patients'},
            
            {"label": "nper", "pattern": [{'LIKE_NUM': True,'OP':'+'},{'ORTH': '%','OP':'+'},{'LOWER': 'diabetes','OP':'+'},{'ORTH': ',','OP':'+'}],'id':'comorbidity percent'},
            {"label": "nper", "pattern": [{'LOWER': 'diabetes','OP':'+'},{'ORTH': '(','OP':'+'},{'LIKE_NUM': True,'OP':'+'},{'IS_PUNCT': True,'OP':'+'},{'LIKE_NUM': True,'OP':'+'},{'IS_PUNCT': True,'OP':'+'}],'id':'comorbidity percent'},
            {"label": "nper", "pattern": [{'LOWER': 'diabetes','OP':'+'},{'ORTH': '(','OP':'+'},{'LIKE_NUM': True,'OP':'+'},{'ORTH': '%','OP':'+'}],'id':'comorbidity percent'},
           ]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

<a id='viz3'></a>
## spaCy In-Line Example
### To help highlight how spaCy identifies entities using the patterns defined above, the "displaCy" capabilities can be used to review the entities "live" in the data itself.

In [ ]:
displacy_example_df=article_selection_df5[article_selection_df5['sha'] =='ad3bfbf5daf646a4ba1be6495c8b83c797944577']
data = []
docs_in = nlp.pipe(displacy_example_df['full_text'].values)
docs_out = []
titles_in = displacy_example_df['title'].values
for i, doc in enumerate(docs_in):
    if len(doc.ents) > 1:
        doc.user_data['title'] = titles_in[i]
        docs_out.append(doc)
        for ent in doc.ents:
            data.append({'id': i,'entity_value': ent.text, 'entity_type': ent.label_})

displacy.render(docs_out, style='ent', page=True, jupyter=True)

### Up to this point, all text that has been extracted or was present within the metadata file is represented as single strings or objects for each row or article.
### In order to determine the key statistics and relevant sentences in which the required data resides, spaCy Sentencizer was used to separate each sentence into distinct rows for each article.
### In addition to ease of use, this helps eliminate the extraction of statistics in later steps that are not relevant or could be taken out of context.

In [ ]:
entity_extraction=[]
docs_in = nlp.pipe(article_selection_df5['full_text'].values)
docs_out = []
sha_in = article_selection_df5['sha'].values
titles_in = article_selection_df5['title'].values
for i, doc in enumerate(docs_in):
    if len(doc.ents) > 1:
        doc.user_data['sha'] = sha_in[i]
        doc.user_data['title'] = titles_in[i]
        docs_out.append(doc)
        for j, sent in enumerate(doc.sents):
            sent_data = {}
            for ent in sent.ents:
                if sent_data.get(ent.label_) is None:
                    sent_data[ent.label_] = ent.text
                else:
                    sent_data[ent.label_] = sent_data[ent.label_] + ';' + ent.text
            entity_extraction.append({'docID': i,'sha':sha_in[i],'title':titles_in[i],'entity_sentencizer':sent.text,'sentID': j, **sent_data})
entity_extraction_df1=pd.DataFrame(entity_extraction)

### In this section, we separate instances where the relevant statistics appear in the full text extract.
### We used the spaCy sentencizer to break the analysis into chunks when identifying entities. In doing so, we determined that not all statistics reside in the same sentence, especially since the json file extracts are not always accurate, complete, and often contain file headers and footers.¶
### The cell below places precedence on those instances where the nFatal and nper statistics are present in single sentences.  Then for those instances where there is no single sentence identified, statistics spanning multiple sentences are extracted and consolidated.

In [ ]:
same_sentence_extract=sql("""select sha,title,sentID, 'SameSentence' as StatsExtraction,nFatal, nper from entity_extraction_df1 
                            where nFatal is not null AND nper is not null""")

diff_sentence_extract=sql("""select a.sha,a.title,a.sentID,'DiffSentence' as StatsExtraction, a.nFatal,b.nper from 
           entity_extraction_df1 a join entity_extraction_df1 b 
           on a.sha=b.sha and a.nFatal is not null and b.nper is not null and (b.sentID>a.sentID AND b.sentID - a.sentID < 10)
          """)
#prefer the same sentence findings, if possible, so removing them from the second dataframe (the multi-sentence results)
diff_sentence_extract=diff_sentence_extract[~diff_sentence_extract['sha'].isin(same_sentence_extract['sha'])]

entity_extraction_df2=same_sentence_extract.append(diff_sentence_extract)

### Authors declare and document statistics differently within their articles. The entities extracted via the logic above can return results in varying order.
### For example, when referring to the number of diabetic patients that were a part of the study cohort, some articles list the data with the order of number of cases then "diabetes" while others use "diabetes" and then list the number.
### Due to the variance in how data is reported, the code in this cell puts the entity values in the same order (comorbidity type then value).

In [ ]:
nper = entity_extraction_df2['nper']
comorbidities = ['diabetes']
metrics = []
como_types = []
for ent in nper:
    for mo in comorbidities:
        if mo in ent:
            como_types.append(mo)
            metrics.append(ent.replace(mo, '').strip())

data_intermediate = {'Comorbidity_Type': como_types, 'metrics': metrics}
df_int = pd.DataFrame(data_intermediate)
entity_cleansing_df1=pd.merge(entity_extraction_df2,df_int,on=nper)
entity_cleansing_df1=entity_cleansing_df1.drop_duplicates()

### In its current state, the above logic did not remove the sha listed in the next cell due to the fact that this article IS in fact a good candidate; the challenge dataset and subsequent json data currently available does not support proper extraction of the statistics needed for this meta-analysis.

In [ ]:
entity_cleansing_df1=entity_cleansing_df1[entity_cleansing_df1.sha != '875b7c463f00772fa0dc18ada678bc1ff16a4274']

### The dataframe from the above step is displayed below to highlight the differences between how authors publish their statistics. Refer to the values in the "nper" column to see how the order for which diabetes and the related values are referenced vary between articles.

In [ ]:
entity_cleansing_df1

### In this section, we separate the number of fatalities from the related entity and remove unnecessary symbols and punctuation so the statistics can be correctly and consistently be extracted from the entities.

In [ ]:
entity_cleansing_df1['Fatalities Number']=entity_cleansing_df1['nFatal'].str.split(" ",expand=True)[0]
#pulling out the diabetes percentages (Some statistics have both the number and %, some are just the %, etc.)
entity_cleansing_df1['Comorbidity Percentage']=entity_cleansing_df1['metrics'].replace(to_replace=r'\([1-9][0-9]/[1-9][0-9]',value='',regex=True).replace(to_replace=r'\(',value='',regex=True).replace(to_replace=r'\)',value='',regex=True).replace(to_replace=r'[1-9][0-9]-',value='',regex=True).replace(to_replace=r'[0-9],',value='',regex=True).replace(to_replace=r',',value='',regex=True).replace(to_replace=r'%',value='',regex=True)
entity_cleansing_df1['Comorbidity Percentage']=entity_cleansing_df1['Comorbidity Percentage'].astype(float).div(100).round(4)

entities_extracted=entity_cleansing_df1[['sha','title','Comorbidity_Type','Fatalities Number','Comorbidity Percentage']].drop_duplicates()

#Converting the Fatalities Number column to int such that the following meta-analysis steps could be executed; previously the dtype was defined as an object
entities_extracted=entities_extracted.astype({'Fatalities Number': 'int32'})

### Since previous comments and code reference the additional clean up required to properly extract and then utilize statistics in the text data, the results displayed below highlight what the "final" dataset looks like when proceeding to the meta-analysis step.

In [ ]:
entities_extracted

## Conducting a Meta-Analysis on the Prevalence of Diabetes in COVID-19 Fatal Cases

In [ ]:
#Create Confidence intervals and standard errors
meta_analysis_df=pd.DataFrame(entities_extracted)
meta_analysis_df['L_95CL'] = meta_analysis_df['Comorbidity Percentage']-(1.96*np.sqrt((meta_analysis_df['Comorbidity Percentage']*(1-meta_analysis_df['Comorbidity Percentage']))/meta_analysis_df['Fatalities Number']))
meta_analysis_df['U_95CL'] = meta_analysis_df['Comorbidity Percentage']+(1.96*np.sqrt((meta_analysis_df['Comorbidity Percentage']*(1-meta_analysis_df['Comorbidity Percentage']))/meta_analysis_df['Fatalities Number']))
meta_analysis_df['SE'] = np.sqrt((meta_analysis_df['Comorbidity Percentage']*(1-meta_analysis_df['Comorbidity Percentage']))/meta_analysis_df['Fatalities Number'])
meta_analysis_df['Study_ID'] = np.arange(len(meta_analysis_df))

In [ ]:
#Compute pooled estimate and 95% CI
total_deaths=sum(meta_analysis_df['Fatalities Number'])
tot_deaths=str(total_deaths)

total_deaths_diab=sum(meta_analysis_df['Fatalities Number']*meta_analysis_df['Comorbidity Percentage'])
tot_deaths_diab=str(round(total_deaths_diab))

prev_diab_deaths= round((total_deaths_diab/total_deaths)*100,2)
pr_diab_deaths=str(prev_diab_deaths)
prev_diab_deaths_num=prev_diab_deaths/100

LCL95= round(prev_diab_deaths_num-(1.96*np.sqrt((prev_diab_deaths_num*(1-prev_diab_deaths_num))/total_deaths)),4)
LCL95_char=str(LCL95*100)
UCL95= round(prev_diab_deaths_num+(1.96*np.sqrt((prev_diab_deaths_num*(1-prev_diab_deaths_num))/total_deaths)),4)
UCL95_char=str(UCL95*100)


print("Total Deaths: " + tot_deaths)
print("Total Deaths with Diabetes: " + tot_deaths_diab)
print("Pooled Prevalence of Diabetes in Fatal Cases: " + pr_diab_deaths + "% , 95% CI: ( " + LCL95_char + "% ," + UCL95_char + "% )" )

<a id='viz4'></a>
### Defining and Displaying the Diabetes Prevalence in Fatal COVID-19 Cases by Study Chart

In [ ]:
# Build the plot
fig, ax = plt.subplots()
ax.bar(meta_analysis_df['Study_ID'], meta_analysis_df['Comorbidity Percentage'], yerr=meta_analysis_df['SE'], align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Proportion of Deaths with Diabetes')
ax.set_xticks(meta_analysis_df['Study_ID'])
ax.set_xticklabels(meta_analysis_df['Study_ID'])
ax.set_xlabel('Study')
ax.set_title('Diabetes Prevalence in Fatal COVID-19 Cases by Study')
ax.yaxis.grid(True)

#Add Horizontal Line for pooled estimate and Confidence Limits
ax.axhline(LCL95, color='green', linewidth=1)
ax.axhline(prev_diab_deaths_num, color='darkgreen', linewidth=3)
ax.axhline(UCL95, color='green', linewidth=1)


## Summary of Findings:
### At the time of submission this notebook identified 6 separate scientific studies and automated the extraction, meta-analyisis, and visualization of diabetes prevalence estimates in fatal COVID-19 cases across all studies. The pooled estimate of diabetes prevalence was calculated and found to be 28.71% (95% CI:23.0%,34.0%). In contrast, 34.2 million Americans, or 10.5% of the population, have diabetes (CDC, 2018). Increased knowledge about the role of diabetes in COVID-19 clinical manifestation and outcomes can help patients and physicians understand risk factors. Policy and public health officials can use this information to plan and distribute resources to respond to COVID-19 in high risk populations. 

## Where do we go from here?
* Utilize the code above to explore the risk of other comorobidities in COVID-19 patients
* Attempt to develop a solution to extract the tables out of select articles
* Re-evaluate ML/AI packages and processes on articles with extracted table data to predict whether new articles added to the challenge fit the use case

### GDIT Contributors: 
#### Colleen Kummet, PhD;  Andrew Horton; Erik Burlin; Badri Prasad, PhD; Yeon Choi, MS; Kristie Chung, MS; Eric Feigl-Ding, PhD 